In [15]:
import pandas as pd
import numpy as np

import glob

In [16]:
#set directory to look in
di = 'C:/Users/geeze/Desktop/'

In [17]:
#get a list of files there
files = glob.glob(di + '*.csv')
show = list(enumerate(files))

In [18]:
#get user input
print('Which file is your post-echo transfer report?')
print(list(enumerate(files)))
num = int(input())

Which file is your post-echo transfer report?
[(0, 'C:/Users/geeze/Desktop\\20200213 1536 plate 1 report.csv')]
0


In [19]:
#open requested file
df = pd.read_csv(files[num])

In [21]:
#munge nicely
col_labels = []
for i in df.index:
    
    #every other row except the one I want to use as the column names has at least one NaN in it. Just get the one with
    #no NaNs
    if df.loc[i].isnull().any():
        pass
    else:
        col_labels.append(i)

In [22]:
#print error if things aren't as expected with the column renaming
if len(col_labels) == 1:
    col_labels = col_labels[0]
else:
    print('more than one potential column label set')

In [23]:
#make labels nice and slice just what you want to see
df = (df.loc[col_labels:].set_axis(df.loc[col_labels], axis=1, inplace=False)
      .drop(col_labels, axis=0)
     .reset_index(drop=True))

df = df.loc[~df['Source Well'].isnull()]

In [24]:
#get list of wells used in protocol
wells = df['Source Well'].unique()
#it doesn't necessarily use every well on each protocol, so you'll need to know approximately how much is in the unused
#ones to find out how much remains in the plate

In [25]:
#find out how much is left in the wells used and estimate what's left in the wells NOT used, then calc total remaining
end_volumes = []
for w in wells:
    d = df.loc[df['Source Well'] == w]
    
    end_vol = min(d['Current Fluid Volume'])
    
    end_volumes.append(float(end_vol))
    
#this assumes you put ~55uL in each echo well, we're conservative and guess we're off by -2 in each
#also assume dead volume of 15uL
remains = np.sum(end_volumes) + 53*(384 - len(wells)) - (384*15)

In [26]:
#output
if remains/(1536*4) > 1.05:
    print('do another, you have {} 1536 plates remaining in source plate'.format(remains/(1536*4)))
else:
    print('refill needed')

do another, you have 1.35446044921875 1536 plates remaining in source plate
